In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive/Streak"

In [ ]:
import requests
from bs4 import BeautifulSoup, Tag, NavigableString
import pandas as pd
import time
from datetime import datetime as dt
from datetime import timedelta as td
import collections
import os
import re

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None) 
pd.set_option('max_colwidth', 220)

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:45.0) Gecko/20100101 Firefox/45.0'}
pd.set_option('display.max_rows', None) 
path="/content/drive/My Drive/Streak/"
path_mem='https://contests.covers.com/Survivor/Contestant/CompletedPicks/'
www='https://contests.covers.com'

files=os.listdir(path)
files=list(filter(lambda x: len(x)==19,files))
files.sort()

# **ЧТЕНИЕ**

In [ ]:
#считываем и формируем текущую таблицу в soup и streak
r=requests.get('https://contests.covers.com/survivor/currentleaderboard',headers=headers)
soup=BeautifulSoup(r.text,'html.parser')

streak=pd.DataFrame(columns=['Rank','Member','Streak','Best','Last Pick','Profile','Deadline'])
st=pd.read_html('https://contests.covers.com/survivor/currentleaderboard')
for i in range(3):
    streak=streak.append(st[i])

In [ ]:
#очищаем данные и находим pending
#curr_streak_pending - датафрейм pending
#pending - список ссылок
dat=(dt.now()).strftime('%b %d')
date=(dt.now()).strftime('%y-%m-%d')
curr_streak=streak.copy()
curr_streak.insert(0,'Date',date)
curr_streak=curr_streak.reset_index()
curr_streak.drop(labels=['Profile','index'],axis=1,inplace=True)
curr_streak_pending=curr_streak.query("Deadline=='Pending'").reset_index()
curr_streak_pending.drop(labels=['index','Deadline'],axis=1,inplace=True)
#находим все ссылки на pending
links=soup.find_all(class_='cmg_qtip_link')
pending=[]
for el in links:
  link=www+el.get('rel')[0]
  pending.append(link)
if len(pending)==len(curr_streak_pending):
  print(dat,'Количество -',len(pending))
  display(curr_streak_pending)
else:
  print('Данные не совпадают!')

Nov 22 Количество - 31


,Date,Rank,Member,Streak,Best,Last Pick
0,20-11-22,1,Absolutxedge22,15,15,"Nov 16, 2020"
1,20-11-22,3,cockyvolfan,10,11,"Nov 22, 2020"
2,20-11-22,5,PSimpson89,9,9,"Nov 21, 2020"
3,20-11-22,6,808rushem,9,9,"Nov 22, 2020"
4,20-11-22,7,hangar18,9,12,"Nov 22, 2020"
5,20-11-22,8,BradsBargains,9,10,"Nov 21, 2020"
6,20-11-22,10,EliSmails,8,8,"Nov 22, 2020"
7,20-11-22,12,Ace909,8,8,"Nov 18, 2020"
8,20-11-22,13,ukula,8,9,"Nov 22, 2020"
9,20-11-22,14,jaydog921,8,8,"Nov 20, 2020"


In [ ]:
#собираем инфу в один датафрейм
#curr_str_pend - все pending со ставками
#curr_day - полная таблица дня для файла
curr_str_pend=pd.DataFrame(columns=['Member','League','BetDate','Matchup','Selection'],index=[i for i in range(len(pending))])

for i,link in enumerate(pending):
  try:
    bet=pd.read_html(link)
    r2=requests.get(link,headers=headers)
    soup2=BeautifulSoup(r2.text,'html.parser')
    inf=soup2.h3.get_text().split(' - ')
    liga=inf[0]
    data=inf[1]
    sel=bet[0].iloc[0]['Selection']+' '+str(bet[0].iloc[0]['Odds'])
    curr_str_pend.iloc[i]['Member']=curr_streak_pending.iloc[i]['Member']

    curr_str_pend.iloc[i]['League']=liga
    curr_str_pend.iloc[i]['BetDate']=data
    curr_str_pend.iloc[i]['Matchup']=bet[0].iloc[0]['Matchup']
    curr_str_pend.iloc[i]['Selection']=sel
    time.sleep(7)
  except ValueError:
    pass
curr_pending=curr_streak_pending.merge(curr_str_pend,on='Member',how='left')
curr_pending.dropna(inplace=True)
curr_pending['Matchup']=list(map(lambda x: x.replace(' vs. ',' at. '),curr_pending['Matchup']))
curr_pending_streak=curr_pending.drop(labels=['Date','Rank','Last Pick','Best'],axis=1)
curr_pending.drop(labels=['Date','Rank','Last Pick','Best','Streak'],axis=1,inplace=True)
curr_day=(curr_streak.merge(curr_pending,on='Member',how='left')).fillna('')
display(curr_pending_streak.sort_values(by=['Streak','League']))
curr_day.to_csv(path+'Streak_'+date+'.csv',index=False)

,Member,Streak,League,BetDate,Matchup,Selection
26,Ruskiy9,6,NFL,Nov 22,DAL at. MIN,DAL 7.5
27,tknowles5,6,NFL,Nov 22,GB at. IND,GB 2.0
28,2pac shabazz,6,NFL,Nov 22,DAL at. MIN,MIN -7.5
29,suecity777,6,NFL,Nov 22,PIT at. JAC,JAC 10.5
30,roblivings,6,NFL,Nov 22,NE at. HOU,NE -2.0
24,built,7,NCAAF,Dec 12,ULM at. ARST,Over 68.0
25,bigmac25,7,NCAAF,Dec 12,PITT at. GT,Over 51.5
17,pigspks7,7,NFL,Nov 22,TEN at. BAL,Under 50.5
18,CTBear,7,NFL,Nov 22,PHI at. CLE,CLE -2.5
19,Pullmyfinger14,7,NFL,Nov 22,ATL at. NO,ATL 3.5


In [ ]:
#ОБНОВЛЕНИЕ СПИСКА ИГРОКОВ. ЗАПУСКАТЬ КАЖДЫЙ ДЕНЬ
today_members=pd.DataFrame()
tmp=pd.DataFrame()
r=requests.get(f'https://contests.covers.com/Survivor/CurrentLeaderboard?pageNum=1',headers=headers)
s=BeautifulSoup(r.text,'html.parser')
l=s.find_all(class_='data-container')
for el1 in l:
  ll=el1.find_all('a')
  for el2 in ll:
    txt=el2.get_text()
    if txt not in ['View','Pending','']:
      link=str(el2.get('href')).split('/')[-1]
      tmp.loc[txt,'Link']=link
  today_members=today_members.append(tmp,sort=False)
  time.sleep(7)
today_members.reset_index(inplace=True)
today_members.head()
today_members.to_csv(path+'AllMembers.csv',index=False,header=False,mode='a')

# **ОБРАБОТКА**

In [ ]:
all_streak=pd.DataFrame()
for el in files:
  st=pd.read_csv(path+el)
  st=st.dropna()
  st.reset_index(drop=True,inplace=True)
  all_streak=all_streak.append(st,sort=False)
display(len(all_streak),all_streak.tail(11))

3741

,Date,Rank,Member,Streak,Best,Last Pick,Deadline,League,BetDate,Matchup,Selection
14,20-10-08,36,kkreider,7,11,"Oct 8, 2020",Pending,NCAAF,Oct 8,TULN at. HOU,HOU -7.0
15,20-10-08,39,navydiver2,7,17,"Oct 8, 2020",Pending,NFL,Oct 8,TB at. CHI,TB -3.5
16,20-10-08,42,zdog6,7,12,"Oct 7, 2020",Pending,NFL,Oct 8,TB at. CHI,TB -4.5
17,20-10-08,43,mc2010,7,21,"Oct 8, 2020",Pending,NBA,Oct 9,MIA at. LAL,LAL -7.0
18,20-10-08,44,elodoya24,7,10,"Oct 8, 2020",Pending,MLB,Oct 8,TB at. NYY,TB 110
19,20-10-08,45,Larrysbills,7,7,"Oct 7, 2020",Pending,NFL,Oct 8,TB at. CHI,Over 44.0
20,20-10-08,46,faustobone,7,12,"Oct 8, 2020",Pending,MLB,Oct 8,ATL at. MIA,Over 8.5
21,20-10-08,47,mcelaj,7,10,"Oct 8, 2020",Pending,MLB,Oct 8,TB at. NYY,NYY -130
22,20-10-08,48,ecoo,7,7,"Oct 8, 2020",Pending,MLB,Oct 8,ATL at. MIA,ATL -130
23,20-10-08,49,305,7,8,"Oct 8, 2020",Pending,MLB,Oct 8,ATL at. MIA,MIA 110


In [ ]:
all_streak['Streak'].value_counts().reset_index().sort_values(by=['index'])

,index,Streak
8,5,38
4,6,198
0,7,1148
1,8,1097
2,9,622
3,10,310
5,11,160
6,12,74
7,13,42
9,14,31


In [ ]:
players=all_streak['Member'].to_list()
#all_streak.query(f"Member=='{players[3]}'")
all_streak.query(f"Streak>9")

In [ ]:
xx=pd.read_csv(path+files[39])
xx

In [ ]:
display(curr_day.sort_values(by=['Streak','Deadline','League'],ascending=False))


In [ ]:
match = collections.Counter()
for el in curr_pending['Matchup']:
  match[el]+=1
vals=list(match.items())
vals.sort(key=lambda x: x[1],reverse=True)

today_picks=pd.DataFrame()
for i in range(len(vals)):
  teams=vals[i][0]
  all_picks=curr_pending.query("Matchup=='{}'".format(teams))
  match = collections.Counter()
  for el in all_picks['Selection']:
    match[el]+=1
  vals2=list(match.items())
  temp=pd.Series([all_picks.iloc[0]['League'],teams,vals2])
  today_picks=today_picks.append(temp.T,ignore_index=True)

today_picks.columns=['League','Matchup','Picks']
today_picks=today_picks.sort_values(by='League')
display(today_picks)

,League,Matchup,Picks
1,NCAAF,USA at. GASO,"[(Under 53.5, 1), (Over 51.5, 1), (USA 3.5, 1)]"
2,NCAAF,CSU at. FRES,"[(CSU -2.5, 1), (Under 59.0, 1), (CSU -1.5, 1)]"
3,NCAAF,KSU at. WVU,"[(KSU 3.0, 1)]"
4,NCAAF,MINN at. MD,"[(MD 19.5, 1)]"
6,NCAAF,PUR at. ILL,"[(PUR -6.5, 1)]"
8,NCAAF,WKU at. BYU,"[(BYU -28.0, 1)]"
0,NFL,ATL at. CAR,"[(CAR -2.0, 1), (Under 49.0, 1), (ATL 2.5, 1), (Over 51.0, 1), (CAR -2.5, 1)]"
5,NFL,SF at. SEA,"[(Over 53.0, 1)]"
7,NFL,NYJ at. KC,"[(KC -20.5, 1)]"
9,NFL,MIN at. GB,"[(GB -6.5, 1)]"


**ОБРАБОТКА**

In [ ]:
#Общий для всех дней
date_start=dt.strptime('Feb 13 2020','%b %d %Y')
date_end=dt.strptime('Mar 05 2020','%b %d %Y')
date_list=[dd.strftime('%y-%m-%d') for dd in pd.date_range(start=date_start,end=date_end)]

date_start=dt.strptime('Mar 07 2020','%b %d %Y')
date_end=dt.strptime('Mar 11 2020','%b %d %Y')
date_list+=[dd.strftime('%y-%m-%d') for dd in pd.date_range(start=date_start,end=date_end)]

date_start=dt.strptime('Aug 02 2020','%b %d %Y')
date_end=dt.now()-td(days=1)
date_list+=[dd.strftime('%y-%m-%d') for dd in pd.date_range(start=date_start,end=date_end)]

for d in range(len(date_list)-1):
  old=pd.read_csv(path+'Streak_{}.csv'.format(date_list[d])).dropna()
  new=pd.read_csv(path+'Streak_{}.csv'.format(date_list[d+1])).dropna()
  played=old.query("Deadline=='Pending'")
  win50=list(set(new['Member']).intersection(played['Member']))
  new50=list(set(new['Member']).difference(old['Member']))
  lose50=list(set(played['Member']).difference(new['Member']))
  print(date_list[d],'-->',date_list[d+1],'|','Выиграли:',len(win50),'Проиграли:',len(lose50),'Разница:',len(lose50)-len(win50))

In [ ]:
def DeadlineCut(row):
  row['Deadline']=row['Deadline'][:7]
  return row
str1=pd.read_csv(path+files[-1])  #текущий день
str2=pd.read_csv(path+files[-2])  #вчерашний день
str1=str1.apply(DeadlineCut,axis=1)
win=str1.query("Deadline=='6  Days'")  #выиграли 


,Date,Rank,Member,Streak,Best,Last Pick,Deadline,League,BetDate,Matchup,Selection
4,20-02-21,5,Messy,10,10,"Feb 20, 2020",6 Days,NaN,NaN,NaN,NaN
6,20-02-21,7,Kstateinkc1,10,10,"Feb 20, 2020",6 Days,NaN,NaN,NaN,NaN
7,20-02-21,8,hearnracing,10,10,"Feb 20, 2020",6 Days,NaN,NaN,NaN,NaN
15,20-02-21,16,VenezolanoVip,8,8,"Feb 19, 2020",6 Days,NaN,NaN,NaN,NaN
21,20-02-21,22,craven768,8,11,"Feb 20, 2020",6 Days,NaN,NaN,NaN,NaN
22,20-02-21,23,sportsking157,8,11,"Feb 20, 2020",6 Days,NaN,NaN,NaN,NaN
25,20-02-21,26,tron501,8,8,"Feb 20, 2020",6 Days,NaN,NaN,NaN,NaN
27,20-02-21,28,book1525,8,9,"Feb 20, 2020",6 Days,NaN,NaN,NaN,NaN
30,20-02-21,31,staynfit247365,7,7,"Feb 20, 2020",6 Days,NaN,NaN,NaN,NaN
34,20-02-21,35,slowgo8day,7,8,"Feb 20, 2020",6 Days,NaN,NaN,NaN,NaN


In [ ]:
#Для вчерашнего дня
dat=(dt.now()).strftime('%y-%m-%d')
dd=dt.now()
dd-=td(days=2)
dat=dd.strftime('%y-%m-%d')
new=pd.read_csv(path+'Streak_'+dat+'.csv')
dd-=td(days=1)
dat=dd.strftime('%y-%m-%d')
old=pd.read_csv(path+'Streak_'+dat+'.csv')
played=old.query("Deadline=='Pending' & BetDate=='{}'".format(dat))
played

,Date,Rank,Member,Streak,Best,Last Pick,Deadline,League,BetDate,Matchup,Selection


In [ ]:
streak_unit=old.append(new)
display(streak_unit['Member'].nunique())
display(streak_unit)

59

,Date,Rank,Member,Last Pick,Deadline,League,BetDate,Matchup,Selection
0,Dec 30,1,sookiS4,"Dec 26, 2019",3 Days 9 Hrs 56 Min,NaN,NaN,NaN,NaN
1,Dec 30,2,CAROLINAFAN23,"Dec 29, 2019",6 Days 8 Hrs 25 Min,NaN,NaN,NaN,NaN
2,Dec 30,3,FCF316,"Dec 29, 2019",6 Days 8 Hrs 25 Min,NaN,NaN,NaN,NaN
3,Dec 30,4,Albichu,"Dec 29, 2019",6 Days 1 Hrs 12 Min,NaN,NaN,NaN,NaN
4,Dec 30,5,Silko,"Dec 30, 2019",Pending,NCAAB,Dec 30,TULN at. MEM,Over 144.0
5,Dec 30,6,RandomWins,"Dec 29, 2019",6 Days 1 Hrs 12 Min,NaN,NaN,NaN,NaN
6,Dec 30,7,kingdave1,"Dec 30, 2019",Pending,NCAAB,Dec 30,YALE at. UNC,UNC -8.0
7,Dec 30,8,DJ1,"Dec 29, 2019",6 Days 1 Hrs,NaN,NaN,NaN,NaN
8,Dec 30,9,komodo44,"Dec 30, 2019",Pending,NCAAB,Dec 30,HOF at. TOWS,HOF 1.0
9,Dec 30,10,Rizzo2332,"Dec 30, 2019",Pending,NCAAF,Dec 30,CAL at. ILL,ILL 6.5


In [ ]:
#new50 - новые игроки
#old50 - старые выигравшие игроки
#lose50 - старые проигравшие игроки
win50=list(set(new['Member']).intersection(played['Member']))
new50=list(set(new['Member']).difference(old['Member']))
old50=list(set(new['Member']).intersection(old['Member']))
lose50=list(set(old['Member']).difference(new['Member']))
display('Выиграли:',len(win50),'Новые:',len(new50),'Проиграли:',len(lose50))

'Выиграли:'

0

'Новые:'

13

'Проиграли:'

13

In [ ]:
losers=old.loc[old['Member'].isin(lose50)].fillna('-')
winners=old.loc[old['Member'].isin(old50)].fillna('-')
losers.query("League!='NCAAF'")
display('Выиграли:',winners.query("League!='-' & BetDate=='{}'".format(dat)),'Проиграли:',losers.query("League!='-' & BetDate=='{}'".format(dat)))

'Выиграли:'

,Date,Rank,Member,Last Pick,Deadline,League,BetDate,Matchup,Selection


'Проиграли:'

,Date,Rank,Member,Last Pick,Deadline,League,BetDate,Matchup,Selection


In [ ]:
new=pd.read_csv(path+'Streak-Nov 25.csv')
old=pd.read_csv(path+'Streak-Nov 26.csv')
pend=old.query("Deadline=='Pending'")
new50=list(set(new['Member']).difference(old['Member']))
win50=list(set(new['Member']).intersection(pend['Member']))
lose50=list(set(pend['Member']).difference(new['Member']))
display('Выиграли:',len(win50),'Проиграли:',len(lose50),'Новые:',len(new50))

'Выиграли:'

1

'Проиграли:'

2

'Новые:'

13

In [ ]:
for el in files:
  df=pd.read_csv(path+el)
  d=el[7:-4]
  df['Date']=d
  df.to_csv(path+el,index=False)

In [ ]:
old.query("Deadline=='Pending'")

,Date,Rank,Member,Streak,Best,Last Pick,Deadline,League,BetDate,Matchup,Selection
2,Nov 21,3,dex,13,13,"Nov 21, 2019",Pending,NCAAB,Nov 21,UTAH vs. CCAR,UTAH -4.5
3,Nov 21,4,BFLO66FAN,12,12,"Nov 21, 2019",Pending,NCAAB,Nov 21,FLA vs. STJOE,Under 150.0
4,Nov 21,5,JHaas2007,12,12,"Nov 20, 2019",Pending,NCAAB,Nov 20,BYU vs. BSU,BSU -1.0
7,Nov 21,8,Aprotipster,9,9,"Nov 20, 2019",Pending,NCAAB,Nov 20,BYU vs. BSU,BSU PK
11,Nov 21,12,govbuc,9,10,"Nov 21, 2019",Pending,NHL,Nov 21,TOR vs. ARI,TOR 100
22,Nov 21,23,chui,8,9,"Nov 20, 2019",Pending,NFL,Nov 21,IND vs. HOU,HOU -3.5
23,Nov 21,24,tog,8,17,"Nov 20, 2019",Pending,NCAAB,Nov 20,SMC vs. FRES,SMC -8.5
26,Nov 21,27,fastNfurious,8,9,"Nov 20, 2019",Pending,NCAAB,Nov 20,BYU vs. BSU,BSU -1.0
30,Nov 21,31,kingpin161,8,9,"Nov 21, 2019",Pending,NFL,Nov 21,IND vs. HOU,IND 3.5
33,Nov 21,34,bradtastic,8,11,"Nov 21, 2019",Pending,NFL,Nov 21,IND vs. HOU,HOU -3.5


In [ ]:
display(old.loc[old['Member'].isin(old50)])
print(' ')
display(old.loc[old['Member'].isin(lose50)])

,Date,Rank,Member,Streak,Best,Last Pick,Deadline,League,BetDate,Matchup,Selection
2,Nov 29,3,flipper7,12,12,"Nov 29, 2019",Pending,NCAAB,Nov 29,DEP vs. MINN,Over 134.0
3,Nov 29,4,conagui,10,10,"Nov 23, 2019",23 Hrs 32 Min,NaN,NaN,NaN,NaN
4,Nov 29,5,OntarioPanthers,10,10,"Nov 29, 2019",6 Days 23 Hrs 11 Min,NaN,NaN,NaN,NaN
5,Nov 29,6,shark58,10,10,"Nov 24, 2019",3 Days 7 Hrs 13 Min,NaN,NaN,NaN,NaN
6,Nov 29,7,ACEhole_1,9,9,"Nov 28, 2019",Pending,NFL,Dec 1,SF vs. BAL,BAL -6.0
7,Nov 29,8,TheGambler34,9,9,"Nov 29, 2019",Pending,NFL,Dec 1,CLE vs. PIT,CLE -2.0
8,Nov 29,9,toxman3434,9,15,"Nov 25, 2019",3 Days 7 Hrs 31 Min,NaN,NaN,NaN,NaN
9,Nov 29,10,packersxbrewers,9,10,"Nov 27, 2019",5 Days 6 Hrs 27 Min,NaN,NaN,NaN,NaN
10,Nov 29,11,magicman1642,9,9,"Nov 29, 2019",Pending,NBA,Nov 29,TOR vs. ORL,TOR -3.5
11,Nov 29,12,LMS387,9,13,"Nov 27, 2019",5 Days 8 Hrs 23 Min,NaN,NaN,NaN,NaN


,Date,Rank,Member,Streak,Best,Last Pick,Deadline,League,BetDate,Matchup,Selection
0,Nov 29,1,B-Rabbit,19,19,"Nov 29, 2019",Pending,NHL,Nov 29,PIT vs. CLB,PIT -125
1,Nov 29,2,Wendigo_49,14,14,"Nov 29, 2019",Pending,NHL,Nov 29,STL vs. DAL,Over 5.0
13,Nov 29,14,Komal77,9,9,"Nov 29, 2019",Pending,NHL,Nov 29,PIT vs. CLB,PIT -120
27,Nov 29,28,robertd21,8,9,"Nov 29, 2019",Pending,NBA,Nov 29,NO vs. OKC,NO 4.0
34,Nov 29,35,vi42baller,8,9,"Nov 28, 2019",6 Days 3 Hrs 59 Min,NaN,NaN,NaN,NaN
36,Nov 29,37,payroll19,7,7,"Nov 29, 2019",Pending,NCAAF,Nov 29,CIN vs. MEM,MEM -13.0
38,Nov 29,39,andylube,7,7,"Nov 29, 2019",Pending,NBA,Nov 29,PHI vs. NY,Over 209.0
47,Nov 29,48,Chaz22,7,8,"Nov 27, 2019",5 Days 7 Hrs 20 Min,NaN,NaN,NaN,NaN


In [ ]:
n50=list(set([1,2,3,4,5]).difference([1,2,3]))
o50=list(set([1,2,3,4,5]).intersection([1,2,3]))

n50

[1, 2, 3]

In [ ]:
#ПЕРЕИМЕНОВАНИЕ ФАЙЛОВ
files=os.listdir(path)
display(files[:2])
for f in files:
  if len(f)==17:
    d=f.split('-')[1].split('.')[0]
    y='19 '
    if 'Jan' in f:
      y='20 '
    dd=dt.strptime(y+d,'%y %b %d')
    dd=dt.strftime(dd,'%y-%m-%d')
    new='Streak_'+dd+'.csv'
    #print(new)
    #os.rename(path+f,path+new)

['Streak_19-11-09.csv', 'Streak_19-11-10.csv']

In [ ]:
#СБОР ВСЕХ ИГРОКОВ
all_members=pd.DataFrame(columns=['Best','Link'])
for p in range(1,70):
  page=pd.DataFrame(columns=['Rank','Member','Streak','Best','Last Pick','Profile','Deadline'])
  tmp=pd.DataFrame(columns=['Best','Link'])
  r=requests.get(f'https://contests.covers.com/Survivor/CurrentLeaderboard?pageNum={p}',headers=headers)
  s=BeautifulSoup(r.text,'html.parser')
  st=pd.read_html(f'https://contests.covers.com/Survivor/CurrentLeaderboard?pageNum={p}')
  for i in range(3):
    page=page.append(st[i])
  best=page['Best'].to_list()
  l=s.find_all(class_='data-container')
  for el1 in l:
    ll=el1.find_all('a')
    for el2 in ll:
      txt=el2.get_text()
      if txt not in ['View','Pending','']:
        link=str(el2.get('href')).split('/')[-1]
        tmp.loc[txt,'Link']=link
  tmp['Best']=best
  all_members=all_members.append(tmp,sort=False)
  time.sleep(7)
all_members.head()
all_members.to_csv(path+'AllMembers2.csv')

In [ ]:
df22['Deadline']

In [ ]:
def corrDeadline(row):
  if 'Days' in row['Deadline']:
    row['Deadline']=row['Deadline'][:7]
  return row
#df1=pd.read_csv(path+files[0])
#df2=pd.read_csv(path+files[1])
#df1=df1.apply(corrDeadline,axis=1)
#df2=df2.apply(corrDeadline,axis=1)

df11=df1.query("Deadline!='Pending'") #не было ставки вчера
df11_nopend=df1.query("Deadline!='Pending'")['Member'].to_list()  #список тех, у кого не было ставки вчера
df22=df2[df2['Member'].isin(df11_nopend)]  #есть в таблицах вчера и сегодня
df33=df22.query("Deadline=='Pending' or Deadline=='6  Days'") #могла быть ставка вчера

display(df33,df11,df22)

,Date,Rank,Member,Streak,Best,Last Pick,Deadline,League,BetDate,Matchup,Selection
2,19-11-10,3,DonnieJr,10,10,"Nov 10, 2019",Pending,NCAAB,Nov 10,CLMB vs. WAKE,CLMB 8.5
4,19-11-10,5,dex,10,13,"Nov 9, 2019",6 Days,NaN,NaN,NaN,NaN
8,19-11-10,9,juiceguru,9,9,"Nov 10, 2019",Pending,NFL,Nov 10,LAR vs. PIT,Over 43.5
13,19-11-10,14,boldtroll,8,9,"Nov 10, 2019",Pending,NFL,Nov 10,MIN vs. DAL,DAL -3.0
14,19-11-10,15,burley,8,11,"Nov 10, 2019",Pending,NFL,Nov 10,CAR vs. GB,GB -4.5
28,19-11-10,29,Abie28,7,9,"Nov 10, 2019",Pending,NFL,Nov 10,MIA vs. IND,MIA 10.5
38,19-11-10,39,SlimEagle,7,9,"Nov 10, 2019",Pending,NFL,Nov 10,ATL vs. NO,NO -13.5


,Date,Rank,Member,Streak,Best,Last Pick,Deadline,League,BetDate,Matchup,Selection
2,19-11-09,3,DonnieJr,10,10,"Nov 6, 2019",4 Days,NaN,NaN,NaN,NaN
3,19-11-09,4,BFLO66FAN,10,11,"Nov 6, 2019",4 Days,NaN,NaN,NaN,NaN
8,19-11-09,9,Kylefields15,9,9,"Nov 8, 2019",6 Days,NaN,NaN,NaN,NaN
10,19-11-09,11,dex,9,13,"Nov 8, 2019",6 Days,NaN,NaN,NaN,NaN
12,19-11-09,13,juiceguru,9,9,"Nov 6, 2019",4 Days,NaN,NaN,NaN,NaN
15,19-11-09,16,makemoney420,8,8,"Nov 8, 2019",6 Days,NaN,NaN,NaN,NaN
16,19-11-09,17,boldtroll,8,9,"Nov 8, 2019",6 Days,NaN,NaN,NaN,NaN
19,19-11-09,20,Harps,8,10,"Nov 7, 2019",5 Days,NaN,NaN,NaN,NaN
27,19-11-09,28,Abie28,7,9,"Oct 28, 2019",1 Days,NaN,NaN,NaN,NaN
28,19-11-09,29,Good2,7,8,"Nov 6, 2019",4 Days,NaN,NaN,NaN,NaN


,Date,Rank,Member,Streak,Best,Last Pick,Deadline,League,BetDate,Matchup,Selection
2,19-11-10,3,DonnieJr,10,10,"Nov 10, 2019",Pending,NCAAB,Nov 10,CLMB vs. WAKE,CLMB 8.5
3,19-11-10,4,BFLO66FAN,10,11,"Nov 6, 2019",3 Days,NaN,NaN,NaN,NaN
4,19-11-10,5,dex,10,13,"Nov 9, 2019",6 Days,NaN,NaN,NaN,NaN
6,19-11-10,7,Kylefields15,9,9,"Nov 8, 2019",5 Days,NaN,NaN,NaN,NaN
8,19-11-10,9,juiceguru,9,9,"Nov 10, 2019",Pending,NFL,Nov 10,LAR vs. PIT,Over 43.5
12,19-11-10,13,makemoney420,8,8,"Nov 8, 2019",5 Days,NaN,NaN,NaN,NaN
13,19-11-10,14,boldtroll,8,9,"Nov 10, 2019",Pending,NFL,Nov 10,MIN vs. DAL,DAL -3.0
14,19-11-10,15,burley,8,11,"Nov 10, 2019",Pending,NFL,Nov 10,CAR vs. GB,GB -4.5
28,19-11-10,29,Abie28,7,9,"Nov 10, 2019",Pending,NFL,Nov 10,MIA vs. IND,MIA 10.5
30,19-11-10,31,Good2,7,8,"Nov 6, 2019",3 Days,NaN,NaN,NaN,NaN


In [ ]:
update_mem=pd.DataFrame()
date=df33['Date'].to_list()[0]
date=dt.strptime(date,'%y-%m-%d')
date=dt.strftime(date,'%b %d, %Y')
for mem in df33['Member'].to_list()[1:2]:
  curr_link=f'https://contests.covers.com/Survivor/CurrentLeaderboard/SearchResults?Username={mem}&ads=false'
  curr_mem=pd.read_html(curr_link)
  best=curr_mem[0]['Best'][0]
  r3=requests.get(curr_link,headers=headers)
  s3=BeautifulSoup(r3.text,'html.parser')
  mem_link=s3.find(class_='follow').get('data-follow-contestantid')
  tmp=pd.DataFrame(pd.Series([mem,best,mem_link])).T
  update_mem=update_mem.append(tmp)
  mem_link=path_mem+mem_link
  print(f'{mem}: {mem_link}')

  mem_link=mem_link+'/100/1'
  rx=requests.get(mem_link,headers=headers)
  sx=BeautifulSoup(rx.text,'html.parser')
  sx1=BeautifulSoup()
  tag1 = Tag(sx1,name='table')
  sx1.insert(0,tag1)
  tag1.insert(0,sx)
  file2 = open(path+'tab.html', "w+", encoding='utf-8')
  file2.write(sx1.prettify())
  file2.close()
  time.sleep(10)
  tab=pd.read_html(path+'tab.html')[0]
  tab.columns=['Date','League','Teams','Score','Pick','Res']
  if date in tab['Date'].to_list():
    print('OK')
display(tab)
#update_mem.to_csv(path+'AllMembers.csv',index=False,header=False,mode='a')

dex: https://contests.covers.com/Survivor/Contestant/CompletedPicks/48932b86-065e-e511-9e61-0024e8753722


,Date,League,Teams,Score,Pick,Res
0,"Mar 12, 2020",NCAAB,CSU Bakersfield Texas Rio Grande Valley,0 0,UTRGV -2.0,Void
1,"Mar 11, 2020",NCAAB,California Stanford,63 51,CAL +9.5,Win
2,"Mar 10, 2020",NBA,Minnesota Houston,111 117,MIN +12.5,Win
3,"Mar 9, 2020",NCAAB,San Francisco Gonzaga,77 81,GONZ -13.0,Loss
4,"Mar 8, 2020",NBA,New Orleans Minnesota,120 107,MIN +6.5,Loss
5,"Mar 7, 2020",NCAAB,Cal Poly SLO UC Santa Barbara,67 69,Over 134.5,Win
6,"Mar 6, 2020",NBA,Miami New Orleans,104 110,NO -1.5,Win
7,"Mar 5, 2020",NBA,Philadelphia Sacramento,125 108,PHI +5.0,Win
8,"Mar 4, 2020",NBA,Oklahoma City Detroit,114 107,OKC -8.0,Loss
9,"Mar 3, 2020",NBA,Brooklyn Boston,129 120,BK +9.0,Win


In [ ]:
if date in tab['Date'].to_list():
    print('OK')

In [ ]:
#ВОССТАНОВЛЕНИЕ ПРОПУСКОВ. 1/4
files=os.listdir(path)
start='Streak_20-09-21.csv'
end='Streak_20-09-23.csv'
files2=end   #files[-5]
start1=start
end1=files[0]

date1=files2[7:-4] #Дата в датафрейме new_mem
date1=dt.strptime(date1,'%y-%m-%d')
#date1=dt.strftime(date1,'%b %d, %Y')
past_st=pd.read_csv(path+start1)
curr_st=pd.read_csv(path+files2)
curr_st=curr_st.drop(['Best','Streak'],axis=1)
curr_st.insert(3,'Best',-1)
curr_st.insert(3,'Streak',-1)
win=list(set(past_st['Member']).intersection(curr_st['Member']))   #игроки есть в обоих списках
for mem in win:
  curr=curr_st.query(f"Member=='{mem}'")
  past=past_st.query(f"Member=='{mem}'")
  curr_date=(curr['Last Pick']).to_list()[0]
  past_date=(past['Last Pick']).to_list()[0]
  ind=curr.index[0]
  if curr_date==past_date:
    curr_st.loc[ind,'Streak']=past['Streak'].to_list()[0]
    curr_st.loc[ind,'Best']=past['Best'].to_list()[0]
  else:
    curr_st.loc[ind,'Streak']=int(past['Streak'].to_list()[0])+1
    if int(past['Best'].to_list()[0])<int(past['Streak'].to_list()[0])+1:
      curr_st.loc[ind,'Best']=int(past['Streak'].to_list()[0])+1
    else:
      curr_st.loc[ind,'Best']=past['Best'].to_list()[0]
curr_st


,Date,Rank,Member,Streak,Best,Last Pick,Deadline,League,BetDate,Matchup,Selection
0,20-09-23,1,rpeake,15,15,"Sep 20, 2020",4 Days 23 Min,NaN,NaN,NaN,NaN
1,20-09-23,2,Cainkoni19,11,11,"Sep 23, 2020",Pending,MLB,Sep 23,DET at. MIN,Under 8.5
2,20-09-23,3,alexander16,10,11,"Sep 22, 2020",6 Days 7 Hrs 42 Min,NaN,NaN,NaN,NaN
3,20-09-23,4,dbacchusdds,11,11,"Sep 18, 2020",2 Days 6 Hrs 50 Min,NaN,NaN,NaN,NaN
4,20-09-23,5,vimbove,10,10,"Sep 21, 2020",5 Days 7 Hrs 34 Min,NaN,NaN,NaN,NaN
5,20-09-23,6,suhuynh,9,9,"Sep 23, 2020",Pending,MLB,Sep 23,DET at. MIN,Over 8.5
6,20-09-23,7,garyline,10,10,"Sep 20, 2020",4 Days 23 Min,NaN,NaN,NaN,NaN
7,20-09-23,8,GWarner27,8,12,"Sep 21, 2020",5 Days 5 Hrs 43 Min,NaN,NaN,NaN,NaN
8,20-09-23,9,venisse,9,13,"Sep 23, 2020",Pending,NBA,Sep 23,BOS at. MIA,Over 210.5
9,20-09-23,10,jprdbulldog20,9,10,"Sep 19, 2020",3 Days 5 Hrs 58 Min,NaN,NaN,NaN,NaN


In [ ]:
win

In [ ]:
# создаем датафрейм новых игроков и собираем ссылки на профайлы. 2/4
new_mem=curr_st.query("Best==-1").reset_index(drop=True)
new_mem.sort_values(by=['Member'],inplace=True)
new_mem_list=new_mem['Member'].to_list()
new_mem.reset_index(inplace=True,drop=True)
all_members=pd.read_csv(path+'AllMembers.csv',index_col=0)
mem_dict={}
update_mem=pd.DataFrame()
for j,mem in enumerate(new_mem_list):
  try:
    mem_link=path_mem+all_members.loc[mem,'Link']
    new_mem.loc[j,'Best']=all_members.loc[mem,'Best']
    print(f'{mem}: {mem_link}')
    mem_dict[mem]=mem_link
  except:
    curr_link=f'https://contests.covers.com/Survivor/CurrentLeaderboard/SearchResults?Username={mem}&ads=false'
    curr_mem=pd.read_html(curr_link)
    best=curr_mem[0]['Best'][0]
    r3=requests.get(curr_link,headers=headers)
    s3=BeautifulSoup(r3.text,'html.parser')
    mem_link=s3.find(class_='follow').get('data-follow-contestantid')
    tmp=pd.DataFrame(pd.Series([mem,best,mem_link])).T
    update_mem=update_mem.append(tmp)
    mem_link=path_mem+mem_link
    print(f'{mem}: {mem_link}')
    mem_dict[mem]=mem_link
display(update_mem)
update_mem.to_csv(path+'AllMembers.csv',index=False,header=False,mode='a')

BGDADDYMOOSE39: https://contests.covers.com/Survivor/Contestant/CompletedPicks/f969df85-065e-e511-9e61-0024e8753722
BabyMoney2020: https://contests.covers.com/Survivor/Contestant/CompletedPicks/4314048e-7f53-4d7e-935e-ab57013d00bf
Banker: https://contests.covers.com/Survivor/Contestant/CompletedPicks/435cdf85-065e-e511-9e61-0024e8753722
CAM1102: https://contests.covers.com/Survivor/Contestant/CompletedPicks/09b3a07c-e912-4fbc-8ddd-aa81010fc297
CoachDaniel: https://contests.covers.com/Survivor/Contestant/CompletedPicks/9bca1448-de98-44b1-bc8a-ac05013da32c
Stburg: https://contests.covers.com/Survivor/Contestant/CompletedPicks/f65ac3af-6503-4322-81c8-a75800f34121
Steph_en: https://contests.covers.com/Survivor/Contestant/CompletedPicks/979d2543-dee9-4d7c-827f-a52a017e7140
TheFriscoKid420: https://contests.covers.com/Survivor/Contestant/CompletedPicks/45c15287-065e-e511-9e61-0024e8753722
Velvetah: https://contests.covers.com/Survivor/Contestant/CompletedPicks/19f36c87-065e-e511-9e61-0024e87

,0,1,2
0,BGDADDYMOOSE39,7,f969df85-065e-e511-9e61-0024e8753722
0,BabyMoney2020,8,4314048e-7f53-4d7e-935e-ab57013d00bf
0,Banker,11,435cdf85-065e-e511-9e61-0024e8753722
0,CAM1102,9,09b3a07c-e912-4fbc-8ddd-aa81010fc297
0,CoachDaniel,8,9bca1448-de98-44b1-bc8a-ac05013da32c
0,Stburg,9,f65ac3af-6503-4322-81c8-a75800f34121
0,Steph_en,10,979d2543-dee9-4d7c-827f-a52a017e7140
0,TheFriscoKid420,8,45c15287-065e-e511-9e61-0024e8753722
0,Velvetah,8,19f36c87-065e-e511-9e61-0024e8753722
0,chasp22,14,76571186-065e-e511-9e61-0024e8753722


In [ ]:
#ОБЩИЙ МОДУЛЬ. 3/4
new_mem=curr_st.query("Best==-1").reset_index(drop=True)
new_mem.sort_values(by=['Member'],inplace=True)
new_mem_list=new_mem['Member'].to_list()
new_mem.reset_index(inplace=True,drop=True)
all_members=pd.read_csv(path+'AllMembers.csv',index_col=0)
mem_dict={}
for j,mem in enumerate(new_mem_list):
  try:
    mem_link=path_mem+all_members.loc[mem,'Link']+'/70/1'
    new_mem.loc[j,'Best']=all_members.loc[mem,'Best']
    print(f'{mem}: {mem_link}')
    mem_dict[mem]=mem_link
    rx=requests.get(mem_link,headers=headers)
    sx=BeautifulSoup(rx.text,'html.parser')
    sx1=BeautifulSoup()
    tag1 = Tag(sx1,name='table')
    sx1.insert(0,tag1)
    tag1.insert(0,sx)
    file2 = open(path+'tab.html', "w+", encoding='utf-8')
    file2.write(sx1.prettify())
    file2.close()
    time.sleep(15)
    tab=pd.read_html(path+'tab.html')[0]
    tab.columns=['Date','League','Teams','Score','Pick','Res']
    
    c=len(tab)-1
    seria=0
    date2=tab.loc[c,'Date']
    date2=dt.strptime(date2,'%b %d, %Y')
    while date2<date1:
      if date2>=date1:
        break
      if tab.loc[c,'Res']=='Win':
        seria+=1
      elif tab.loc[c,'Res']=='Loss':
        seria=0
      else:
        pass
      c-=1
      date2=tab.loc[c,'Date']
      date2=dt.strptime(date2,'%b %d, %Y')
      new_mem.loc[j,'Streak']=seria
      #print(c,seria)
  except:
    print(mem)
display(new_mem)

BGDADDYMOOSE39
BabyMoney2020
Banker
CAM1102
CoachDaniel
Stburg
Steph_en
TheFriscoKid420
Velvetah
chasp22
jkasem
kennyj
kipster319
riz818
rquiroz
suncash
thebigmg
tknowles5


,Date,Rank,Member,Streak,Best,Last Pick,Deadline,League,BetDate,Matchup,Selection
0,20-09-23,43,BGDADDYMOOSE39,-1,-1,"Sep 21, 2020",5 Days 7 Hrs 34 Min,NaN,NaN,NaN,NaN
1,20-09-23,47,BabyMoney2020,-1,-1,"Sep 20, 2020",4 Days 30 Min,NaN,NaN,NaN,NaN
2,20-09-23,50,Banker,-1,-1,"Sep 22, 2020",Pending,NHL,Sep 23,TB at. DAL,Over 5.0
3,20-09-23,27,CAM1102,-1,-1,"Sep 21, 2020",5 Days 7 Hrs 45 Min,NaN,NaN,NaN,NaN
4,20-09-23,18,CoachDaniel,-1,-1,"Sep 22, 2020",6 Days 5 Hrs 42 Min,NaN,NaN,NaN,NaN
5,20-09-23,39,Stburg,-1,-1,"Sep 23, 2020",Pending,MLB,Sep 23,TEX at. ARI,Under 9.5
6,20-09-23,25,Steph_en,-1,-1,"Sep 22, 2020",6 Days 7 Hrs 42 Min,NaN,NaN,NaN,NaN
7,20-09-23,40,TheFriscoKid420,-1,-1,"Sep 21, 2020",5 Days 7 Hrs 34 Min,NaN,NaN,NaN,NaN
8,20-09-23,49,Velvetah,-1,-1,"Sep 19, 2020",3 Days 7 Hrs 20 Min,NaN,NaN,NaN,NaN
9,20-09-23,35,chasp22,-1,-1,"Sep 21, 2020",5 Days 7 Hrs 34 Min,NaN,NaN,NaN,NaN


In [ ]:
#ПЕРЕЗАПИСЫВАЕМ ФАЙЛ ДНЯ. 4/4
curr_st=curr_st.query("Best!=-1")
curr_st=curr_st.append(new_mem,sort=False)
curr_st.to_csv(path+files2,index=False)

In [ ]:
f11=pd.read_csv(path+'Streak_20-02-11.csv')
f13=pd.read_csv(path+'Streak_20-02-13.csv')
f12_mem=list(set(f13['Member']).intersection(f11['Member']))  #не проиграли 12.02
f12=f11.query(f"Member in {f12_mem}") #DF непроигравших
f12['Date']='20-02-12'
f12.to_csv(path+'Streak_20-02-12.csv',index=False)
f12

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Date,Rank,Member,Streak,Best,Last Pick,Deadline,League,BetDate,Matchup,Selection
0,20-02-12,1,mrellis1972,13,13,"Feb 10, 2020",6 Days 8 Hrs 17 Min,NaN,NaN,NaN,NaN
2,20-02-12,3,m33w99,12,12,"Feb 10, 2020",6 Days 10 Hrs 1 Min,NaN,NaN,NaN,NaN
4,20-02-12,5,rillim,11,11,"Feb 10, 2020",6 Days 8 Hrs 16 Min,NaN,NaN,NaN,NaN
5,20-02-12,6,themorphy,11,11,"Feb 6, 2020",2 Days 9 Hrs 20 Min,NaN,NaN,NaN,NaN
7,20-02-12,8,Therizjc,11,11,"Feb 8, 2020",4 Days 6 Hrs 58 Min,NaN,NaN,NaN,NaN
9,20-02-12,10,spiders96,10,15,"Feb 8, 2020",4 Days 23 Hrs 10 Min,NaN,NaN,NaN,NaN
10,20-02-12,11,Lscott0311,10,10,"Feb 10, 2020",6 Days 6 Hrs 18 Min,NaN,NaN,NaN,NaN
11,20-02-12,12,jjbills,10,12,"Feb 6, 2020",2 Days 8 Hrs 28 Min,NaN,NaN,NaN,NaN
12,20-02-12,13,whodat005,10,12,"Feb 10, 2020",6 Days 8 Hrs 17 Min,NaN,NaN,NaN,NaN
13,20-02-12,14,shanebr22,9,9,"Feb 11, 2020",Pending,NBA,Feb 11,POR at. NO,Over 240.0


In [ ]:
def new(row):
  if row['Deadline'][0]=='6':
    return row
f13_new=f13.apply(new,axis=1) #новички и старички - играли 12.02 и не проиграли
f13_new.dropna(how='all',inplace=True)
f13_new['Rank']=list(map(int,f13_new['Rank']))
f13_new['Streak']=list(map(int,f13_new['Streak']))
f13_new['Best']=list(map(int,f13_new['Best']))
f13_mem=list(set(f13_new['Member']).difference(f12['Member'])) #только новички - играли 12.02 и не проиграли
f12=f13_new.query(f"Member in {f13_mem}") #DF непроигравших
f12['Date']='20-02-12'
f12.to_csv(path+'Streak_20-02-12.csv',index=False,mode='a',header=False)
f12

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,Date,Rank,Member,Streak,Best,Last Pick,Deadline,League,BetDate,Matchup,Selection
24,20-02-12,25,annekim520,8,8,"Feb 12, 2020",6 Days 6 Hrs 51 Min,NaN,NaN,NaN,NaN
26,20-02-12,27,CLewis1990,8,8,"Feb 12, 2020",6 Days 6 Hrs 52 Min,NaN,NaN,NaN,NaN
27,20-02-12,28,DizBSpeedy,8,13,"Feb 12, 2020",6 Days 7 Hrs 34 Min,NaN,NaN,NaN,NaN
30,20-02-12,31,crich1055,8,14,"Feb 12, 2020",6 Days 5 Hrs 42 Min,NaN,NaN,NaN,NaN
31,20-02-12,32,Getty3,8,9,"Feb 11, 2020",6 Days 9 Hrs 36 Min,NaN,NaN,NaN,NaN
32,20-02-12,33,mruck,8,10,"Feb 12, 2020",6 Days 9 Hrs 27 Min,NaN,NaN,NaN,NaN
33,20-02-12,34,LuckyGuy,8,10,"Feb 12, 2020",6 Days 6 Hrs 27 Min,NaN,NaN,NaN,NaN
41,20-02-12,42,sego,8,10,"Feb 12, 2020",6 Days 7 Hrs 23 Min,NaN,NaN,NaN,NaN
42,20-02-12,43,Cabo-Rob,8,8,"Feb 12, 2020",6 Days 6 Hrs 27 Min,NaN,NaN,NaN,NaN
43,20-02-12,44,VenezolanoVip,7,7,"Feb 12, 2020",6 Days 9 Hrs 27 Min,NaN,NaN,NaN,NaN


In [ ]:
display(pd.read_csv(path+'Streak_20-03-02.csv'))

In [ ]:
#СБОР НАЗВАНИЙ КОМАНД
def skobki(teams):
  pat=re.compile('\(\d+\)')
  x=re.search(pat,teams)
  if x:
    rem=teams[x.start():x.end()]
    teams=teams.replace(rem,'')
  x=re.search(pat,teams)
  if x:
    rem=teams[x.start():x.end()]
    teams=teams.replace(rem,'')
  return teams

date='2020-03-02'
d=dt.strptime(date,'%Y-%m-%d')
NHL={}
for i in range(5):
  r=requests.get(f'https://www.covers.com/Sports/NHL/Matchups?selectedDate={date}',headers=headers)
  s=BeautifulSoup(r.text,'html.parser')
  s1=s.find_all(class_='cmg_matchup_header_team_names')
  s2=s.find_all(class_='cmg_team_name')
  for c in range(len(s1)):
    full_names=s1[c].get_text().replace('\r\n','').replace('\n\n','').strip()
    if '(' in full_names:
      full_names=skobki(full_names)
    full_names=full_names.split(' at ')
    short_names=(s2[c*2].get_text()+s2[c*2+1].get_text()).split('\r\n')
    short_names=[short_names[1].strip(),short_names[3].strip()]
    #display(full_names,short_names)
    NHL[full_names[0].strip()]=short_names[0]
    NHL[full_names[1].strip()]=short_names[1]
  d-=td(days=1)
  date=dt.strftime(d,'%Y-%m-%d')

df_NHL=pd.DataFrame(columns=['Full','Short'])
df_NHL['Full']=NCAAB.keys()
df_NHL['Short']=NCAAB.values()
df_NHL.to_csv(path+'Teams_NHL.csv',index=False)

In [ ]:
NHL

{'Anaheim': 'ANA',
 'Arizona': 'ARI',
 'Boston': 'BOS',
 'Buffalo': 'BUF',
 'Calgary': 'CAL',
 'Carolina': 'CAR',
 'Chicago': 'CHI',
 'Colorado': 'COL',
 'Columbus': 'CLB',
 'Dallas': 'DAL',
 'Detroit': 'DET',
 'Edmonton': 'EDM',
 'Florida': 'FLA',
 'Los Angeles': 'LA',
 'Minnesota': 'MIN',
 'Montreal': 'MON',
 'NY Islanders': 'NYI',
 'NY Rangers': 'NYR',
 'Nashville': 'NAS',
 'New Jersey': 'NJ',
 'Ottawa': 'OTT',
 'Philadelphia': 'PHI',
 'Pittsburgh': 'PIT',
 'San Jose': 'SJ',
 'St. Louis': 'STL',
 'Tampa Bay': 'TB',
 'Toronto': 'TOR',
 'Vancouver': 'VAN',
 'Vegas': 'VEG',
 'Washington': 'WAS',
 'Winnipeg': 'WIN'}